In [1]:
import numpy as np
import tensorflow  as tf
from google.colab import files
from google.colab import drive
import glob
import pickle
import random
import itertools as itr
import librosa
from tensorflow.contrib.layers import fully_connected

import warnings
warnings.filterwarnings('ignore')

In [2]:
drive.mount('/content/Drive/',force_remount =  True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive/


In [0]:
dirpath = "Drive/My Drive/Acads/Deep Learning/A4/"

In [0]:
#Loading pickle files
trs = open(dirpath+"hw4_trs.pkl", "rb")
trs = pickle.load(trs)

In [0]:
#Loading pickle files
tes = open(dirpath+"hw4_tes.pkl", "rb")
tes = pickle.load(tes)

In [0]:
# trs_spec_abs = []
# for f in trs:
#   S=librosa.stft(f, n_fft=1024, hop_length=512)
  
#   #Taking absolute of input and output of training file
#   S =  S.T
#   S_abs = np.abs(S)
  
#   trs_spec_abs.append(S_abs)
  
# print(len(trs_spec_abs))

In [0]:
def gen_STFT(RawAmpVsTime):

  spectrogram = librosa.stft(RawAmpVsTime, n_fft=1024, hop_length=512)
  spectrogram = np.transpose(spectrogram)
  spectrogram_Mag = np.abs(spectrogram)
  
  return spectrogram,spectrogram_Mag

In [0]:
def getSTFT(AllRawAmpVsTime):
  STFTs = []
  
  for i in AllRawAmpVsTime:
    spectrogram,spectrogram_Mag = gen_STFT(i)
    STFTs.append(spectrogram_Mag)

  return STFTs

In [0]:
trs_spec_abs = getSTFT(trs)
tes_spec_abs = getSTFT(tes)

In [0]:
# tes_spec_abs = []
# for f in tes:
#   S=librosa.stft(f, n_fft=1024, hop_length=512)
  
#   #Taking absolute of input and output of training file
#   S =  S.T
#   S_abs = np.abs(S)
  
#   tes_spec_abs.append(S_abs)
  
# print(len(tes_spec_abs))

In [11]:
#checking shape of spectrograms
trs_spec_abs[0].shape

(32, 513)

In [12]:
#checking shape of spectrograms
tes_spec_abs[0].shape

(45, 513)

In [13]:
len(trs_spec_abs)

500

In [0]:
def PositiveExamples(SpeakerIndices,sampleNum):
  PositivePairs = list(itr.combinations(SpeakerIndices,2))
  if sampleNum is not None:
     PositivePairs =  random.sample(PositivePairs,sampleNum)
   
  
  return PositivePairs

def NegativeExamples(SpeakerIndices,RestSpeakerIndices, sampleNum):
    NegativePairs = list(itr.product(SpeakerIndices,RestSpeakerIndices))
    
    if sampleNum is not None:
      NegativePairs =  random.sample(NegativePairs,sampleNum)
    
    return NegativePairs

In [0]:
def genAllSpeaker(Total_speakers, Data ):
  Data = np.array(Data)
  Total_clips,Time_Stamp , Features = Data.shape
  
  AudioPerSpeaker = int(Total_clips//Total_speakers)
  
  AllIndices = list(range(Total_clips))
  
  trainLeft = []
  trainRight = []
  Y = []
  for iteration in range(0,Total_clips,10):
    SpeakerIndices = list(range(iteration,iteration+10))
    
    positivePairs = PositiveExamples(SpeakerIndices,45)
    
    RestSpeakerIndices = np.delete(AllIndices,SpeakerIndices)
    
    
    negativePairs = NegativeExamples(SpeakerIndices,RestSpeakerIndices,45)
    
    positiveLeftIndices, positiveRightIndices = [i[0] for i in positivePairs] , [i[1] for i in positivePairs] 
    negativeLeftIndices, negativeRightIndices = [i[0] for i in negativePairs] , [i[1] for i in negativePairs] 
    
    posPlusNegXLeft = np.concatenate((positiveLeftIndices, negativeLeftIndices), axis=None)
    posPlusNegXRight = np.concatenate((positiveRightIndices, negativeRightIndices), axis=None)
    
    YBatch = [1 for i in range(len(positiveLeftIndices))] + [0 for i in range(len(negativeLeftIndices))] 
    YBatch = np.array(YBatch)    
    Y.append(YBatch)
    leftNetworkExamples  =  Data[posPlusNegXLeft,:,:] 
    rightNetworkExamples =  Data[posPlusNegXRight,:,:] 
    trainLeft.append(leftNetworkExamples)
    trainRight.append(rightNetworkExamples)
    
  return trainLeft,trainRight,Y

In [0]:
minibatch_left,minibatch_right,y_actual = genAllSpeaker(50,trs_spec_abs)
minibatch_left_test,minibatch_right_test,y_test = genAllSpeaker(10,tes_spec_abs)

In [0]:
# minibatch_left=[]
# minibatch_right=[]
# for i in range(50):
#   left=[]
#   right=[]
#   speaker = trs_spec_abs[10*i:i*10+10]
#   pos = list(itertools.combinations(speaker,2))
#   for j in range(45):
#     left.append(pos[j][0])
#     right.append(pos[j][1])
   
#   neg_ind = list(range(0,10*i))+list(range(i*10+10,500))
#   neg_ind_sampled = np.random.choice(neg_ind, size=45, replace=False)
  
#   minibatch_neg=[]
#   for k,ind in enumerate(neg_ind_sampled):
#     pos = speaker[k%10]
#     neg = trs_spec_abs[ind]
#     left.append(pos)
#     right.append(neg)
    
#   minibatch_left.append(left)
#   minibatch_right.append(right)

  

In [0]:
# minibatch_left = np.array(minibatch_left)
# minibatch_right = np.array(minibatch_right)
# minibatch_right.shape

In [0]:
# Xl = np.reshape(minibatch_left, [50*90, 32, 513])
# Xr = np.reshape(minibatch_right, [50*90, 32, 513])

In [0]:
# y_actual = [([1 for i in range(45)]+[-1 for i in range(45)]) for i in range(50)]
# y_actual = np.array(y_actual)
# y_actual.shape

In [0]:
# y_train = np.reshape(y_actual, [50*90])

In [0]:
X_left = tf.placeholder(tf.float32, [None, None, 513])
X_right = tf.placeholder(tf.float32, [None, None, 513])

y = tf.placeholder(tf.float32, [None])
time_domain = tf.placeholder(tf.int32)


In [0]:
# def ip_reshape(X, time_domain):
#   global x_image_2d
#   x_image_2d = tf.reshape(X, [-1,time_domain,513])
#   return x_image_2d
  

In [0]:
num_units = [513]

In [0]:
def NN(X,reuse,td):
  cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n, reuse = reuse) for n in num_units]
  stacked_lstm = tf.nn.rnn_cell.MultiRNNCell(cells)
  rnn_outputs, state = tf.nn.dynamic_rnn(stacked_lstm, X, dtype = tf.float32)
  hidden1 = tf.layers.dense(rnn_outputs, 513, name="hidden1", activation=tf.nn.tanh, reuse = reuse)
  drop_out = tf.nn.dropout(hidden1, keep_prob = 0.9)
  outputs = tf.layers.dense(drop_out, 513, name="hidden", activation=tf.nn.relu, reuse = reuse)
  outputs = tf.reshape(hidden1,shape = [-1,td*513])
  return outputs

In [20]:
out1 = NN(X_left,False,time_domain)
out2 = NN(X_right,True,time_domain)
dotProduct = tf.reduce_sum(tf.multiply(out1,out2),axis = 1 ,name = 'dotproduct')
yPred = tf.nn.sigmoid(dotProduct)

binarisedOutput = tf.cast(tf.math.greater(yPred,0.5), tf.int16)


accuracy = tf.metrics.accuracy(labels = y,predictions = binarisedOutput)

lossCalcu = tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = dotProduct)
lossCalcu  = tf.reduce_sum(lossCalcu)

gradOptimizer = tf.train.AdamOptimizer(learning_rate=0.0005)

train = gradOptimizer.minimize(lossCalcu)
# accuracy = RSquared(LastOutput,Y)

init = tf.global_variables_initializer()



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating

In [0]:
#Using Adam Optimizer for minimizing the loss
lr = 0.0005
optimizer2 = tf.train.AdamOptimizer(lr).minimize(lossCalcu)

init = tf.global_variables_initializer()

In [22]:
#Training the NN
n_epochs = 100
acc_2d = []

saver = tf.train.Saver()
with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    epoch_loss=0
    for iteration in range(len(minibatch_left)):
  #     for i in range(2):
      X_batch_left = minibatch_left[iteration]#[45*i:45*i+45]
      X_batch_right = minibatch_right[iteration]#[45*i:45*i+45]
      y_batch = y_actual[iteration]#[45*i:45*i+45]

      sess.run(optimizer2, feed_dict = {X_left: X_batch_left, X_right: X_batch_right, y: y_batch, time_domain:minibatch_left[0].shape[1]})

      l = lossCalcu.eval(feed_dict={X_left: X_batch_left, X_right: X_batch_right, y: y_batch, time_domain:minibatch_left[0].shape[1]})
      epoch_loss+=l
    if epoch%10==0:
      print('epoch:', epoch+1, 'epoch loss',epoch_loss)
      
      save_path = saver.save(sess, "/tmp/Q1.ckpt")



epoch: 1 epoch loss 13891.512020111084
epoch: 11 epoch loss 558.8830995559692
epoch: 21 epoch loss 155.59236297011375
epoch: 31 epoch loss 2.3235499737784266
epoch: 41 epoch loss 0.79826001659967
epoch: 51 epoch loss 0.4065752741880715
epoch: 61 epoch loss 0.23936660622712225
epoch: 71 epoch loss 0.15213845699327067
epoch: 81 epoch loss 0.10134414126514457
epoch: 91 epoch loss 0.06965036803740077


### Checking accuracy on test

In [0]:
# minibatch_left_test=[]
# minibatch_right_test=[]
# for i in range(20):
#   left=[]
#   right=[]
#   speaker = tes_spec_abs[10*i:i*10+10]
#   pos = list(itertools.combinations(speaker,2))
#   for j in range(45):
#     left.append(pos[j][0])
#     right.append(pos[j][1])
   
#   neg_ind = list(range(0,10*i))+list(range(i*10+10,200))
#   neg_ind_sampled = np.random.choice(neg_ind, size=45, replace=False)
  
#   minibatch_neg=[]
#   for k,ind in enumerate(neg_ind_sampled):
#     pos = speaker[k%10]
#     neg = tes_spec_abs[ind]
#     left.append(pos)
#     right.append(neg)
    
#   minibatch_left_test.append(left)
#   minibatch_right_test.append(right)

  

In [0]:
# minibatch_left_test = np.array(minibatch_left_test)
# minibatch_right_test = np.array(minibatch_right_test)
# minibatch_right_test.shape

In [0]:
# y_test = [([1 for i in range(45)]+[-1 for i in range(45)]) for i in range(20)]
# y_test = np.array(y_test)
# y_test.shape

In [0]:
Xl = np.array(minibatch_left_test).reshape((-1,45,513))
Xr = np.array(minibatch_right_test).reshape((-1,45,513))
y_test = np.array(y_test).reshape(-1)

In [24]:
y_test.shape

(1800,)

In [0]:
# test_pred = sess.run(yPred, feed_dict={X_left:Xl, X_right:Xr, y:y_test, time_domain:Xl.shape[1]})

In [26]:
with tf.Session() as sess:
  init.run()
  saver.restore(sess, "/tmp/Q1.ckpt")
  test_pred = sess.run(yPred, feed_dict={X_left:Xl, X_right:Xr, y:y_test, time_domain:Xl.shape[1]})

  
print('Accuracy on test data:')
print(sum(test_pred == y_test)/y_test.shape)

INFO:tensorflow:Restoring parameters from /tmp/Q1.ckpt
Accuracy on test data:
[0.36722222]
